# Module 6 Challenge: Vacation Itinerary

### Importing needed modules

In [1]:
import pandas as pd
import gmaps
import requests

from config import g_key

### Importing CSV derived from WeatherPy_Database

In [2]:
city_data_df = pd.read_csv("weather_data/WeatherPy_vacation.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Talnakh,RU,69.49,88.40,57.20,82,40,17.90,scattered clouds,0.0,0.0
1,1,Prado,BR,-17.34,-39.22,69.93,92,24,3.18,few clouds,0.0,0.0
2,2,Salalah,OM,17.02,54.09,84.20,89,90,4.70,overcast clouds,0.0,0.0
3,3,Clyde River,CA,70.47,-68.59,35.60,93,90,4.70,overcast clouds,0.0,0.0
4,4,Vestmannaeyjar,IS,63.44,-20.27,48.88,83,100,10.67,overcast clouds,0.0,0.0


### Choosing four cities that a customer might want to visit based on country input by proximity

In [3]:
country = str(input("Which Country would like to visit? (Alpha-2 code, eg: JP, US, GB): "))
min_temp = float(input("What is the minimum temperature you would like for your trip?: "))
max_temp = float(input("What is the maximum temperature you would like for your trip?: "))

selected_country_df = city_data_df.loc[(city_data_df["Country"] == country) & (city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]

selected_city_1 = selected_country_df.iloc[0]
selected_city_2 = selected_country_df.iloc[1]
selected_city_3 = selected_country_df.iloc[2]
selected_city_4 = selected_country_df.iloc[3]

selected_city_4

Which Country would like to visit? (Alpha-2 code, eg: JP, US, GB): JP
What is the minimum temperature you would like for your trip?: 60
What is the maximum temperature you would like for your trip?: 90


City_ID                                        445
City                                          Naze
Country                                         JP
Lat                                          28.37
Lng                                         129.48
Max Temp                                        77
Humidity                                        94
Cloudiness                                      75
Wind Speed                                   10.29
Current Description    light intensity shower rain
Rain (inches)                                    0
Snow (inches)                                    0
Name: 445, dtype: object

### Getting the latitude-longitude pairs for each city DataFrame as tuples

In [4]:
coord_city1 = (selected_city_1[3],selected_city_1[4])
coord_city2 = (selected_city_2[3],selected_city_2[4])
coord_city3 = (selected_city_3[3],selected_city_3[4])
coord_city4 = (selected_city_4[3],selected_city_4[4])

print(coord_city1, coord_city2, coord_city3, coord_city4)

(32.69, 128.84) (35.73, 140.83) (31.46, 131.23) (28.37, 129.48)


### Creating selected cities dataframe using .iloc function

In [5]:
selected_country_df = selected_country_df.iloc[0:4]
selected_country_df["Hotel Name"] = ""

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

for index, row in selected_country_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels = requests.get(base_url, params=params).json()
    try:
        selected_country_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

selected_country_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches),Hotel Name
42,42,Fukue,JP,32.69,128.84,75.2,73,40,11.41,scattered clouds,0.0,0.0,Goto Guesthouse Utojuku & SA Car Rental
55,55,Hasaki,JP,35.73,140.83,79.0,83,75,14.99,light intensity shower rain,0.0,0.0,Inubosaki Kanko Hotel
288,288,Kushima,JP,31.46,131.23,80.6,61,40,13.87,scattered clouds,0.0,0.0,Hanaoka
445,445,Naze,JP,28.37,129.48,77.0,94,75,10.29,light intensity shower rain,0.0,0.0,奄美ポートタワーホテル


### Plotting into gmaps with markers

In [14]:
gmaps.configure(api_key=g_key)

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in selected_country_df.iterrows()]

locations = selected_country_df[["Lat", "Lng"]]
fig = gmaps.figure()
c1_to_c4 = gmaps.directions_layer(coord_city1, coord_city4, waypoints=[coord_city2, coord_city3], show_markers=False)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(c1_to_c4)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))